In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.stats as st
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
#import pyarrow.parquet as pq
import glob
import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

['part-00000-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00001-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00002-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00003-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00004-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00005-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00006-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00007-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00008-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00009-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00010-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00011-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00012-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00013-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00014-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00015-7508f931-eb2f-4e4e-916d-5dee35182a0c-c000.csv', 'part-00016-7508f931-eb2f-4e4e-916d-5de

In [4]:
allFiles1 = glob.glob("./input/*.csv")
train = pd.DataFrame()
i=1
for file_ in allFiles1:
        df=pd.read_csv(file_)
        #df = pq.read_table(file_).to_pandas()
        train=pd.concat([train,df])
        if i%10==0:
            print(i)
        i+=1
        
del df
train.shape

10
20
30
40


(1458643, 156)

In [ ]:
#learning rate decay


import random
from sklearn.cross_validation   import StratifiedKFold
from sklearn.metrics import mean_squared_error
from math import sqrt

# --------------------------------------------------------------------------------
#
def evalscore(preds, dtrain):
    
    labels = dtrain.get_label()
    
    return 'rmse', sqrt(mean_squared_error(labels, preds))

# --------------------------------------------------------------------------------
#

np.random.seed( 12345 )
random.seed(    1234 )

Train, Test = train_test_split(train, test_size = 0.2)
#del train
X_train = Train.drop(['log_trip_duration'], axis=1).values
y_train = Train["log_trip_duration"].values
del Train
X_test = Test.drop(['log_trip_duration'], axis=1).values
y_test = Test["log_trip_duration"].values
del Test
dtrain = xgb.DMatrix(X_train, label=y_train)
del X_train 
dvalid = xgb.DMatrix(X_test, label=y_test)
del X_test 

va_scores    = []
train_scores = []                    
y_pred              = []

params = {
            'min_child_weight'      : 5,
            "max_depth"             : 15, 
            "eta"                   : 0.5,
            "min_eta"               : 0.00001,
            "eta_decay"             : 0.5,
            "max_fails"             : 5,
            "early_stopping_rounds" : 50,
            "objective"             : 'reg:linear',
            "subsample"             : 0.9, 
            "colsample_bytree"      : 0.9,
            "n_jobs"                : -1,
            "n_estimators"          : 10000, 
            "silent"                : 1,
             'lambda'               : 1000000,
            'booster' :            'gbtree'
    }


num_round       = params["n_estimators"]
eta             = params["eta"]
min_eta         = params["min_eta"]
eta_decay       = params["eta_decay"]
early_stop      = params["early_stopping_rounds"]
max_fails       = params["max_fails"]

params_copy     = dict(params)

total_rounds        = 0
best_rounds         = 0
pvalid              = None
model               = None
best_train_score    = None
best_va_score       = None
fail_count          = 0
best_rounds         = 0
best_model          = None

while eta >= min_eta:           

        model        = xgb.train( params_copy.items(), 
                                  dtrain, 
                                  num_round, 
                                  [(dtrain, 'train'), (dvalid,'valid')], 
                                  early_stopping_rounds=early_stop,
                                  feval=evalscore,
                                  verbose_eval=1,
                                )

        rounds          = model.best_iteration + 1
        total_rounds   += rounds

        train_score = sqrt(mean_squared_error( y_train, model.predict(dtrain, ntree_limit=rounds) ))
        va_score    = sqrt(mean_squared_error( y_test, model.predict(dvalid, ntree_limit=rounds) ))

        if best_va_score is None or va_score > best_va_score:
            fail_count = 0
            best_train_score = train_score
            best_va_score    = va_score
            best_rounds      = rounds
            best_model       = model

            ptrain           = best_model.predict(dtrain, ntree_limit=rounds, output_margin=True)
            pvalid           = best_model.predict(dvalid, ntree_limit=rounds, output_margin=True)

            dtrain.set_base_margin(ptrain)
            dvalid.set_base_margin(pvalid)
        else:
            fail_count += 1

            if fail_count >= max_fails:
                print('max fail reach')
                break

        eta                 = eta_decay * eta
        print("new eta: "+str(eta))
        params_copy["eta"]  = eta

train_scores.append(best_train_score)
va_scores.append(best_va_score)

print("%9.6f : %9.6f" % ( best_train_score, best_va_score ))
  


print("\nWith decay ...\n")   



[0]	train-rmse:4.50952	valid-rmse:4.51093	train-rmse:4.50952	valid-rmse:4.51093
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[1]	train-rmse:3.39661	valid-rmse:3.39795	train-rmse:3.39661	valid-rmse:3.39795
[2]	train-rmse:2.58198	valid-rmse:2.58323	train-rmse:2.58198	valid-rmse:2.58323
[3]	train-rmse:1.99181	valid-rmse:1.99291	train-rmse:1.99181	valid-rmse:1.99291
[4]	train-rmse:1.56989	valid-rmse:1.57088	train-rmse:1.56989	valid-rmse:1.57088
[5]	train-rmse:1.27351	valid-rmse:1.27446	train-rmse:1.27351	valid-rmse:1.27446
[6]	train-rmse:1.06295	valid-rmse:1.06375	train-rmse:1.06295	valid-rmse:1.06375
[7]	train-rmse:0.914163	valid-rmse:0.91477	train-rmse:0.914163	valid-rmse:0.91477
[8]	train-rmse:0.810051	valid-rmse:0.810602	train-rmse:0.810051	valid-rmse:0.810602
[9]	train-rmse:0.733203	valid-rmse:0.733693	train-rmse:0.733202	valid-rmse:0.733693
[10]	train-rmse:0.676726	valid-rmse:0.677169	

In [ ]:
import pickle
pickle.dump(best_model, open("pima.pickle.dat", "wb"))

In [ ]:
print(rounds)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 50))
xgb.plot_importance(final_model, max_num_features=2000,ax=ax)
plt.show()

In [ ]:
name=['id','f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146', 'f147', 'f148', 'f149', 'f150', 'f151', 'f152', 'f153', 'f154']

allFiles1 = glob.glob("./input/nyctaxi-cluster-test-v2/*.csv")
test = pd.DataFrame()
i=1
for file_ in allFiles1:
        df = pd.read_csv(file_,names=name,header=0)
#        df = pq.read_table(file_).to_pandas()
        test=pd.concat([test,df])
        if i%10==0:
            print(i)
        i+=1
        
del df

In [ ]:
test=test.reset_index(drop=True)
test_id=test['id']
test_data=test.drop(['id'], axis=1)
dtest = xgb.DMatrix(test_data)
pred = best_model.predict(dtest,ntree_limit=rounds)
pred = np.exp(pred) - 1
submission = pd.concat([test_id, pd.DataFrame(pred)],axis=1)
submission.columns = ['id','trip_duration']
#submission['trip_duration'] = submission.apply(lambda x : 1 if (x['trip_duration'] <= 0) else x['trip_duration'], axis = 1)
submission.to_csv("submission.csv", index=False)